In [1]:
%matplotlib inline
import os,random

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["KERAS_FLAGS"]  = "device=gpu%d"%(0)

import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
import matplotlib.pyplot as plt
import seaborn as sns
import cPickle, random, sys, keras


Using TensorFlow backend.


In [2]:
import h5py
filename = '../data_cwt.hdf5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())


Keys: [u'Classes', u'LabelClass', u'LabelSnr', u'Snrs', u'X_samples', u'Y_samples']


In [5]:
X = f['X_samples']
Y = f['Y_samples']


In [6]:
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
train_idx = np.random.choice(range(0,int(n_examples)), size=int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))

In [21]:
img_width ,img_height,img_dim =  X[0].shape[0],X[0].shape[1],X[0].shape[2]
print img_width ,img_height ,img_dim

224 224 3


In [22]:
from keras import applications
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, img_dim))
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

80117760/80134624 [============================>.] - ETA: 0s

In [23]:
NUMBER_OF_CLASSES = 11
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(NUMBER_OF_CLASSES, activation="softmax")(x)

In [26]:
from keras.models import Sequential, Model 

# creating the final model 
model_final = Model(input = model.input, output = predictions)

/home/yoni/repos/university/ENV/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  after removing the cwd from sys.path.


In [28]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = keras.optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
batch_size = 16
epochs = 50

def samples_generator(x_list,y_list,index_list,batch_size):
    
    